In [1]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)
from huggingface_hub import login

login(
  #token="Your token",  
  add_to_git_credential=True
)


'\nlogin(\n  #token="Your token",  \n  add_to_git_credential=True\n)\n'

In [3]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
from transformers import StoppingCriteria, StoppingCriteriaList
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

In [ ]:
dataset = load_dataset("json", data_files="/workspace/train_dataset.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
test_dataset = dataset["test"].select(range(20))

## with generate function

In [47]:
for i in test_dataset:
    input_text = i['messages'][0]['content']
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids, 
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 생성된 텍스트를 디코딩합니다
    generated_text = tokenizer.decode(output[0], 
                                      #skip_special_tokens=True
                                     )
    print(generated_text)

<bos>음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.

**동아리 친구들과 다른 동아리 친구들 사이에 벽이 생기는 건 뭐가 문제인가요?**

동아리 마다 다른 활동을 하는 것이고, 그 활동을 통해 동아리 친구들은 공통된 경험을 쌓고 서로 친밀해지는 것은 자연스러운 일이에요. 그렇지만, 다른 동아리 친구들과의 교류를 차단하는 것은 여러 가지 문제를 야기할 수 있어요.

**1. 시야가 좁아지는 문제:**

동아리 친구들과의 관계만을 중시할 경우, 외부 세계와의 교류가 줄어들어 시야가 좁아질 수 있어요. 다른 동아리 친구들은 다양한 관심사와 경험을 가지고 있어, 그들의 이야기와 생각을 통해 새로운 정보와 관점을 얻을 수 있는 기회를 놓칠 수 있어요.

**2. 소통과 이해가 부족해지는 문제:**

다른 동아리 친구들과의 교류를 제한하면, 서로 다른 세계관과 가치관을 이해하는 기회가 줄어들어 소통과 이해가 부족해질 수 있어요. 이는 학교 생활과 사회생활에서의 갈등으로 이어질 수도 있어요.

**3. 긍정적 관계 형성의 기회를 놓치는 문제:**

다른 동아리 친구들과의 교류를 통해 좋은 친구를 사귈 수 있는 기회를 놓쳐요. 

**4. 개방성과 융통성이 저해되는 문제:**

다른 동아리 친구들과의 관계를 경계하는 태도는 개방성과 융통성을 저해하여, 새로운 사람들과의 관계 형성에 어려움을 겪을 수 있어요.

**결론적으로,** 동아리 친구들과의 우정을 소중히 여기는 것은 좋지만, 다른 동아리 친구들과의 교류를 차단하는 것은 학교 생활과 사회생활에서 많은 문제를 야기할 수 있다는 것을 기억해야 해요. 다양한 사람들과 교류하며 시야를 넓히고, 이해와 존중을 바탕으로 


KeyboardInterrupt: 

## with pipeline

In [48]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=512,
    temperature=0.7,
    
)

In [55]:
responses = []
for i in test_dataset[:12]:
    input_text = i['messages'][0]['content']
    output = pipe(
        input_text ,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id
    )
    
    print(output[0]["generated_text"])
    responses.append(output[0]["generated_text"])


 

동아리 활동을 하면서 다른 동아리 친구들과 친해지지 않아도 될까요?

혹시 그런 이야기를 들었으면 친구들에게 **"내가 다른 동아리 친구들과 친해지고 싶어"** 라고 솔직하게 이야기해보는 게 어떨까요? 

친구들이 당신의 진심에 맞춰 생각해줄 거예요. 




혹시, 어떤 질문을 해달라고 하는 건가요?

내가 무엇을 알고 싶은지 말해주시면 최선을 다해 답변할게요. 😊

 팀원들과의 갈등이 심화되고, 업무 효율도 떨어지고 있습니다. 😥

혹시 이런 상황에서 어떻게 해결할 수 있을지 조언 주실 수 있나요?

**팀원들과의 갈등 해결을 위한 몇 가지 조언을 드릴게요.**

**1. 문제의 근본 원인 파악하기**

* 혹시 **구체적으로 어떤 부분에서 갈등이 발생하는지** 자세히 생각해보세요. 
* **업무 분담**, **의견 충돌**, **소통 방식 문제**, **개인적인 문제** 등 원인을 파악하는 것이 중요합니다.

**2. 자기 성찰하기**

*  **나의 행동이나 태도가 갈등을 악화시키고 있지는 않은지** 솔직하게 돌아보세요. 
* 객관적인 시각으로 자신을 평가하고 개선할 부분이 있는지 확인하는 시간을 가지세요.

**3. 솔직하고 직접적인 소통 시도하기**

* 갈등을 잠재우거나 회피하는 것은 문제 해결을 늦추거나 악화시킬 수 있습니다. 
* **대화를 통해 서로의 입장을 이해하고 공감**하는 것이 중요합니다. 
* 감정적인 표현은 피하고 **객관적인 사실과 구체적인 예시를 바탕으로** 대화를 이끌어 나가세요.

**4. 타협점 찾기**

* **모든 것을 자신의 뜻대로 하려고 하기보다는 서로에게 만족할 수 있는 타협점을 찾으려는 노력**이 필요합니다. 
* **완벽한 해답보다는 현실적인 해결 방안**을 모색하는 것이 중요합니다.

**5. 팀워크 강화 노력하기**

* 팀원들과 함께 **팀 목표를 설정하고 달성**하기 위한 노력을 통해 팀워크를 향상시킬 수 있습니다. 
* 긍정적인 분위기를 조성하고 서로에게 도움을 주고받는 문화를 만들어나가세요.
*

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




몇 주 전 짧은 메시지만 보내고 자연스러운 방식으로 연락을 끊었어요. 그때는 아무런 문제가 없는 것처럼 보였는데요.

하지만 이번에 몇 주 동안 연락이 전혀 없어서 걱정이 됩니다. 

전화도 안 돼요, 문자도 안 돼요. 소셜 미디어에도 접속이 없는 것 같아요.

동생이 갑자기 연락이 닿지 않을 정도로 어떤 일이 생겼을지 걱정되고 무슨 일을 해야 할지 모르겠어요.

**지금까지 어떤 노력을 해 보셨나요?**

더 자세한 정보를 알려주시면 더 도움이 될 수 있습니다.

예를 들어:

* 동생의 성별, 나이, 직업 등
* 최근에 어떤 일을 했다는 메시지가 있었나요?
* 동생의 평소 성향 (외향적인가, 내향적인가, 혼자 있는 것을 좋아하는가 등)
* 동생이 가지고 있는 질병이나 약물 복용 여부
* 연락이 닿지 않는 기간 동안 동생의 집이나 주변에서 어떤 변화가 있었는지 
* 동생이 어떤 친구나 가족이 있는지

이러한 정보를 바탕으로 더 구체적인 조언을 드릴 수 있습니다.


**다음과 같은 방법들을 시도해 볼 수 있습니다.**

* **가족이나 친구에게 연락하여 동생의 소식을 묻고, 함께 찾아보는 것을 제안합니다.**
* **동생이 자주 이용하는 곳 (회사, 카페, 학교 등)에 연락하여 동생을 대해 본 적이 있는지 확인합니다.**
* **동생의 주소를 알고 있다면, 가까운 주변의 사람들에게 동생을 봤는지 물어봅니다.**
* **경찰서에 신고하여 실종 신고를 합니다.** 

동생의 안전을 위해 꾸준히 노력하시고, 희망을 잃지 마세요. 



편입생이라 새로운 환경에 적응하는 게 쉽지 않을 거예요. 특히 신입생들이 활기차게 어울리고 있는 모습을 보면서 더 외로움을 느낄 수 있을 거예요. 하지만 걱정 마세요!  친해지는 방법은 분명히 있습니다. 

**1. 먼저 스스로에게 다가가보세요:**

* **자신의 장점을 파악하고 자신감을 가지세요.** 
   좋아하는 취미가 있거나, 뛰어난 부분이 있다면 그것을 바탕으로 다른 사람들과 대화의 시작을 만들어 보세요. 
* **새

KeyboardInterrupt: 

In [59]:
import os 
import pickle
os.makedirs('result', exist_ok=True)
with open('result'+"/responses_before_train.pkl","wb") as f:
    pickle.dump(responses, f)

In [61]:
list_ex_load

[' \n\n동아리 활동을 하면서 다른 동아리 친구들과 친해지지 않아도 될까요?\n\n혹시 그런 이야기를 들었으면 친구들에게 **"내가 다른 동아리 친구들과 친해지고 싶어"** 라고 솔직하게 이야기해보는 게 어떨까요? \n\n친구들이 당신의 진심에 맞춰 생각해줄 거예요. \n\n',
 '\n\n혹시, 어떤 질문을 해달라고 하는 건가요?\n\n내가 무엇을 알고 싶은지 말해주시면 최선을 다해 답변할게요. 😊\n',
 ' 팀원들과의 갈등이 심화되고, 업무 효율도 떨어지고 있습니다. 😥\n\n혹시 이런 상황에서 어떻게 해결할 수 있을지 조언 주실 수 있나요?\n\n**팀원들과의 갈등 해결을 위한 몇 가지 조언을 드릴게요.**\n\n**1. 문제의 근본 원인 파악하기**\n\n* 혹시 **구체적으로 어떤 부분에서 갈등이 발생하는지** 자세히 생각해보세요. \n* **업무 분담**, **의견 충돌**, **소통 방식 문제**, **개인적인 문제** 등 원인을 파악하는 것이 중요합니다.\n\n**2. 자기 성찰하기**\n\n*  **나의 행동이나 태도가 갈등을 악화시키고 있지는 않은지** 솔직하게 돌아보세요. \n* 객관적인 시각으로 자신을 평가하고 개선할 부분이 있는지 확인하는 시간을 가지세요.\n\n**3. 솔직하고 직접적인 소통 시도하기**\n\n* 갈등을 잠재우거나 회피하는 것은 문제 해결을 늦추거나 악화시킬 수 있습니다. \n* **대화를 통해 서로의 입장을 이해하고 공감**하는 것이 중요합니다. \n* 감정적인 표현은 피하고 **객관적인 사실과 구체적인 예시를 바탕으로** 대화를 이끌어 나가세요.\n\n**4. 타협점 찾기**\n\n* **모든 것을 자신의 뜻대로 하려고 하기보다는 서로에게 만족할 수 있는 타협점을 찾으려는 노력**이 필요합니다. \n* **완벽한 해답보다는 현실적인 해결 방안**을 모색하는 것이 중요합니다.\n\n**5. 팀워크 강화 노력하기**\n\n* 팀원들과 함께 **팀 목표를 설정하고 달성**하기 위한 노력을 통해 팀워크를 향상시킬 수 